In [ ]:
# ControlNet with GIFs & Artistic Features
!pip install diffusers transformers accelerate torch torchvision xformers gradio controlnet_aux
!pip install opencv-python numpy Pillow imageio
!pip install --upgrade transformers diffusers

import cv2
import torch
import gradio as gr
import numpy as np
import imageio
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, DPMSolverMultistepScheduler
from diffusers import StableDiffusionInstructPix2PixPipeline  # For style variations
from controlnet_aux import HEDdetector, OpenposeDetector, CannyDetector
from transformers import BlipProcessor, BlipForConditionalGeneration
import random
from datetime import datetime
from typing import List, Tuple

print("✅ AI Circus packages installed!")

# ====================
# 1. ARTIST STYLE LIBRARY
# ====================

class ArtisticStyleLibrary:
    """Library of artistic styles with prompt enhancements"""

    def __init__(self):
        self.styles = {
            "salvador_dali": {
                "name": "Salvador Dalí",
                "prompt_suffix": ", surrealism, melting clocks, dreamlike, distorted reality, detailed painting, surrealist art, by Salvador Dalí",
                "negative_prompt": "realistic, normal, symmetrical, clear edges",
                "conditioning_scale": 0.7,
                "color_palette": "gold, blue, beige, surreal colors"
            },
            "moebius": {
                "name": "Moebius (Jean Giraud)",
                "prompt_suffix": ", sci-fi line art, intricate details, clean lines, vibrant colors, geometric patterns, comic book art, by Moebius",
                "negative_prompt": "blurry, messy, realistic, photographic",
                "conditioning_scale": 0.8,
                "color_palette": "electric blue, magenta, black, white"
            },
            "miyazaki": {
                "name": "Hayao Miyazaki",
                "prompt_suffix": ", Studio Ghibli style, whimsical, magical, soft lighting, fantasy creatures, detailed painting, anime art style",
                "negative_prompt": "dark, scary, realistic, adult themes",
                "conditioning_scale": 0.6,
                "color_palette": "pastel colors, soft greens, sky blue, warm yellows"
            },
            "h_r_giger": {
                "name": "H.R. Giger",
                "prompt_suffix": ", biomechanical, alien, organic machinery, dark surrealism, xenomorph style, detailed horror art",
                "negative_prompt": "cute, happy, bright, simple",
                "conditioning_scale": 0.9,
                "color_palette": "black, metallic, bone white, dark blue"
            },
            "van_gogh": {
                "name": "Vincent van Gogh",
                "prompt_suffix": ", impressionist brushstrokes, swirling patterns, expressive colors, post-impressionism, oil painting texture",
                "negative_prompt": "clean lines, digital art, smooth",
                "conditioning_scale": 0.5,
                "color_palette": "swirling blues, yellows, expressive brushstrokes"
            },
            "bosh": {
                "name": "Hieronymus Bosch",
                "prompt_suffix": ", grotesque fantasy, medieval art, hellish creatures, intricate details, surreal nightmare, triptych style",
                "negative_prompt": "modern, simple, minimal, cute",
                "conditioning_scale": 0.85,
                "color_palette": "earth tones, dark reds, greens, browns"
            }
        }

        # Style-specific model weights (we'll use different seeds for variations)
        self.style_seeds = {
            "salvador_dali": 4242,
            "moebius": 5151,
            "miyazaki": 6161,
            "h_r_giger": 7171,
            "van_gogh": 8181,
            "bosh": 9191
        }

    def get_style_prompt(self, base_prompt: str, style_key: str) -> str:
        """Enhance prompt with artistic style"""
        if style_key in self.styles:
            style = self.styles[style_key]
            return f"{base_prompt} {style['prompt_suffix']}"
        return base_prompt

    def get_backstory(self, creature_name: str, style_key: str, components: List[str] = None) -> str:
        """Generate creative backstory for the creature"""

        backstory_templates = {
            "salvador_dali": [
                f"The '{creature_name}' emerges from a dreamscape where time flows like honey. {random.choice(['Its form shifts with the viewer\'s perception', 'It exists in multiple states simultaneously'])}.",
                f"Known as '{creature_name}', this being defies conventional anatomy. {random.choice(['It dances to silent melodies only it can hear', 'Its very presence warps the fabric of reality around it'])}."
            ],
            "moebius": [
                f"The '{creature_name}' hails from the crystalline dimension of {random.choice(['Aether-X7', 'The Chroma Nebula'])}. Its {random.choice(['geometric patterns', 'energy conduits'])} channel cosmic frequencies.",
                f"'{creature_name}' navigates the {random.choice(['floating archipelagos', 'neural networks'])} of the digital wilderness. {random.choice(['Its circuits glow with ancient knowledge', 'It communicates through light harmonies'])}."
            ],
            "miyazaki": [
                f"Little '{creature_name}' guards the {random.choice(['Whispering Woods', 'Cloud Gardens'])}, where {random.choice(['it helps lost spirits find their way', 'it collects memories in its translucent body'])}.",
                f"A gentle guardian of the {random.choice(['Forbidden Valley', 'Starfall Meadow'])}, '{creature_name}' {random.choice(['rides moonbeams', 'weaves dreams from spider silk'])} each night."
            ],
            "h_r_giger": [
                f"The '{creature_name}' evolved in the {random.choice(['acidic biomes', 'mechanical womb'])} of a dying planet. Its {random.choice(['chitinous plates', 'organic pistons'])} pulse with dark energy.",
                f"'{creature_name}' is a biomechanical horror from the {random.choice(['Xenomorph hive', 'Engineer ruins'])}. {random.choice(['It absorbs technology to evolve', 'Its scream disrupts electrical systems'])}."
            ],
            "van_gogh": [
                f"'{creature_name}' swirls into existence from {random.choice(['a starry night', 'sunflower fields'])}. Its form is an expression of {random.choice(['cosmic loneliness', 'vibrant madness'])}.",
                f"This living brushstroke called '{creature_name}' dances across {random.choice(['wheat fields', 'asylum walls'])}, leaving trails of {random.choice(['iridescent oil', 'starlight'])}."
            ],
            "bosh": [
                f"The '{creature_name}' dwells in the {random.choice(['Garden of Earthly Delights', 'Last Judgment'])}, where {random.choice(['it punishes the sinful with ironic transformations', 'it embodies seven deadly sins in one form'])}.",
                f"'{creature_name}' crawls from the {random.choice(['Hellmouth', 'Alchemical Forge'])}, a {random.choice(['fusion of animal, vegetable, and mineral', 'walking condemnation of human folly'])}."
            ]
        }

        # Hybrid backstory if components provided
        if components and len(components) >= 2:
            base_template = random.choice([
                f"A hybrid of {components[0]} and {components[1]}, '{creature_name}' embodies the tension between {random.choice(['nature and machine', 'dream and reality', 'order and chaos'])}.",
                f"Born from the union of {components[0]} and {components[1]}, '{creature_name}' {random.choice(['carries traits of both yet is neither', 'exists in the liminal space between their worlds'])}."
            ])

            if style_key in backstory_templates:
                style_line = random.choice(backstory_templates[style_key]).split('. ')[-1]
                return f"{base_template} {style_line}"

        # Regular backstory
        if style_key in backstory_templates:
            return random.choice(backstory_templates[style_key])

        return f"The '{creature_name}' is a mysterious being whose origins are unknown."

# ====================
# 2. INTERACTIVE GIF GENERATOR
# ====================

class CreatureAnimator:
    """Create animated GIFs from generated creatures"""

    def __init__(self):
        self.frame_count = 12  # For 2.5 second GIF at ~5fps

    def create_morphing_frames(self, base_image: Image.Image,
                              prompt: str,
                              control_image: Image.Image,
                              pipe,
                              seed: int) -> List[Image.Image]:
        """Generate morphing animation frames"""

        frames = [base_image]

        for i in range(1, self.frame_count):
            # Vary parameters for animation
            current_seed = seed + i * 100
            current_scale = 0.7 + (i % 3) * 0.1  # Oscillating scale

            # Add motion prompt
            motion_words = ["gently moving", "slowly rotating", "pulsing softly",
                          "shimmering", "breathing", "shifting form"]
            motion_prompt = f"{prompt}, {random.choice(motion_words)}"

            # Generate frame with slight variations
            generator = torch.Generator(device="cuda").manual_seed(current_seed)

            result = pipe(
                prompt=motion_prompt,
                image=control_image,
                negative_prompt="static, frozen, still image, video frame",
                num_inference_steps=20,
                guidance_scale=7.0,
                width=512,
                height=512,
                controlnet_conditioning_scale=current_scale,
                generator=generator
            )

            frames.append(result.images[0])

        return frames

    def create_breathing_effect(self, frames: List[Image.Image]) -> List[Image.Image]:
        """Add breathing/pulsing effect to frames"""

        enhanced_frames = []

        for i, frame in enumerate(frames):
            # Create slight zoom/breathing effect
            scale_factor = 1.0 + 0.02 * np.sin(i * np.pi / 3)  # Gentle oscillation

            if scale_factor > 1.0:
                # Slight zoom in
                new_size = int(512 * scale_factor)
                zoomed = frame.resize((new_size, new_size), Image.LANCZOS)
                # Crop back to 512x512
                left = (new_size - 512) // 2
                top = (new_size - 512) // 2
                cropped = zoomed.crop((left, top, left + 512, top + 512))
                enhanced_frames.append(cropped)
            else:
                enhanced_frames.append(frame)

        return enhanced_frames

    def create_gif(self, frames: List[Image.Image],
                  output_path: str = "creature_animation.gif",
                  duration: int = 200) -> str:
        """Save frames as looping GIF"""

        # Ensure frames are same size
        frames = [frame.resize((512, 512), Image.LANCZOS) for frame in frames]

        # Create GIF
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=duration,  # milliseconds per frame
            loop=0,  # Infinite loop
            optimize=True
        )

        return output_path

    def create_wiggle_3d(self, base_image: Image.Image) -> List[Image.Image]:
        """Create simple 3D wiggle effect from single image"""

        frames = []
        base_array = np.array(base_image)

        for i in range(self.frame_count):
            offset = int(5 * np.sin(i * np.pi / 3))

            # Create stereo offset effect
            frame = base_array.copy()
            if offset > 0:
                # Shift red channel
                frame[:, offset:, 0] = base_array[:, :-offset, 0]  # Red channel
                frame[:, :, 1] = base_array[:, :, 1]  # Green stays
                frame[:, :-offset, 2] = base_array[:, offset:, 2]  # Blue channel
            else:
                frame[:, :512+offset, 0] = base_array[:, -offset:, 0]
                frame[:, :, 1] = base_array[:, :, 1]
                frame[:, -offset:, 2] = base_array[:, :512+offset, 2]

            frames.append(Image.fromarray(frame))

        return frames

# ====================
# 3. HYBRID CREATURE GENERATOR
# ====================

class HybridCreatureGenerator:
    """Create hybrid creatures from multiple images"""

    def __init__(self, pipe):
        self.pipe = pipe

    def blend_edge_maps(self, image1: Image.Image,
                       image2: Image.Image,
                       blend_mode: str = "average") -> Image.Image:
        """Blend canny edges from two images"""

        def get_edges(img):
            img_np = np.array(img.convert('L'))
            edges = cv2.Canny(img_np, 100, 200)
            return edges

        edges1 = get_edges(image1)
        edges2 = get_edges(image2)

        if blend_mode == "average":
            blended = (edges1 * 0.5 + edges2 * 0.5).astype(np.uint8)
        elif blend_mode == "maximum":
            blended = np.maximum(edges1, edges2)
        elif blend_mode == "interlace":
            # Create checkerboard pattern
            blended = np.zeros_like(edges1)
            blended[::2, ::2] = edges1[::2, ::2]
            blended[1::2, 1::2] = edges2[1::2, 1::2]

        # Convert to 3-channel
        blended_3d = np.stack([blended] * 3, axis=-1)

        return Image.fromarray(blended_3d)

    def create_component_labels(self, image1_desc: str,
                               image2_desc: str,
                               result_image: Image.Image) -> Image.Image:
        """Create labeled diagram showing component origins"""

        labeled = result_image.copy()
        draw = ImageDraw.Draw(labeled)

        # Add labels
        draw.text((20, 20), f"← From: {image1_desc}",
                 fill=(255, 200, 100), stroke_width=2, stroke_fill=(0, 0, 0))
        draw.text((20, 480), f"← From: {image2_desc}",
                 fill=(100, 200, 255), stroke_width=2, stroke_fill=(0, 0, 0))
        draw.text((200, 240), "HYBRID CREATION",
                 fill=(255, 255, 255), stroke_width=3, stroke_fill=(0, 0, 0))

        # Add arrows
        draw.line([(50, 40), (150, 150)], fill=(255, 200, 100), width=3)
        draw.line([(50, 460), (150, 350)], fill=(100, 200, 255), width=3)

        return labeled

# ====================
# 4. MAIN PIPELINE
# ====================

class EnhancedAICircus:
    """Main AI Circus system"""

    def __init__(self):
        print("🎪 Initializing AI Circus...")

        # Load ControlNet
        self.controlnet = ControlNetModel.from_pretrained(
            "lllyasviel/sd-controlnet-canny",
            torch_dtype=torch.float16
        )

        # Load pipeline
        self.pipe = StableDiffusionControlNetPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            controlnet=self.controlnet,
            torch_dtype=torch.float16,
            safety_checker=None,
            requires_safety_checker=False
        )
        self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(self.pipe.scheduler.config)
        self.pipe = self.pipe.to("cuda")

        # Initialize components
        self.style_library = ArtisticStyleLibrary()
        self.animator = CreatureAnimator()
        self.hybrid_generator = HybridCreatureGenerator(self.pipe)

        # Initialize image captioning for automatic descriptions
        try:
            self.caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
            self.caption_model = BlipForConditionalGeneration.from_pretrained(
                "Salesforce/blip-image-captioning-base",
                torch_dtype=torch.float16
            ).to("cuda")
            self.has_captioning = True
        except:
            print("⚠️ Could not load captioning model, using manual descriptions")
            self.has_captioning = False

        print("✅ AI Circus initialized!")

    def auto_caption_image(self, image: Image.Image) -> str:
        """Automatically generate description for an image"""
        if not self.has_captioning:
            return "mysterious creature"

        try:
            inputs = self.caption_processor(image, return_tensors="pt").to("cuda", torch.float16)
            out = self.caption_model.generate(**inputs, max_length=50)
            caption = self.caption_processor.decode(out[0], skip_special_tokens=True)
            return caption.lower()
        except:
            return "interesting subject"

    def generate_creature(self,
                         uploaded_image: Image.Image,
                         prompt: str,
                         style_key: str = "moebius",
                         bend_intensity: float = 0.3,
                         seed: int = 42,
                         generate_gif: bool = True) -> dict:
        """Generate creature with artistic style and optional animation"""

        try:
            # Preprocess image
            uploaded_image = uploaded_image.convert('RGB').resize((512, 512), Image.LANCZOS)

            # Get edges
            img_np = np.array(uploaded_image.convert('L'))
            edges = cv2.Canny(img_np, 100, 200)
            edges_3d = np.stack([edges] * 3, axis=-1)
            control_image = Image.fromarray(edges_3d)

            # Apply artistic style
            style = self.style_library.styles.get(style_key, self.style_library.styles["moebius"])
            enhanced_prompt = self.style_library.get_style_prompt(prompt, style_key)

            # Generate creature
            style_seed = self.style_library.style_seeds.get(style_key, seed)
            generator = torch.Generator(device="cuda").manual_seed(style_seed)

            # Apply network bending (gentle)
            for name, param in self.pipe.controlnet.named_parameters():
                if 'conv' in name and param.requires_grad:
                    noise = torch.randn_like(param) * bend_intensity * 0.005
                    param.data += noise

            result = self.pipe(
                prompt=enhanced_prompt,
                image=control_image,
                negative_prompt=style["negative_prompt"],
                num_inference_steps=25,
                guidance_scale=7.5,
                width=512,
                height=512,
                controlnet_conditioning_scale=style["conditioning_scale"],
                generator=generator
            )

            creature_image = result.images[0]

            # Generate backstory
            creature_name = prompt.split(',')[0].replace('a ', '').replace('an ', '').title()
            backstory = self.style_library.get_backstory(creature_name, style_key)

            # Generate GIF if requested
            gif_path = None
            if generate_gif:
                frames = self.animator.create_morphing_frames(
                    creature_image, enhanced_prompt, control_image, self.pipe, seed
                )
                frames = self.animator.create_breathing_effect(frames)
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                gif_path = f"creature_{timestamp}.gif"
                self.animator.create_gif(frames, gif_path, duration=150)

            return {
                "creature_image": creature_image,
                "control_image": control_image,
                "backstory": backstory,
                "gif_path": gif_path,
                "style_name": style["name"],
                "color_palette": style["color_palette"]
            }

        except Exception as e:
            print(f"❌ Generation error: {e}")
            return self._create_error_response(str(e))

    def generate_hybrid_creature(self,
                                image1: Image.Image,
                                image2: Image.Image,
                                desc1: str = None,
                                desc2: str = None,
                                prompt: str = "a hybrid creature",
                                style_key: str = "moebius",
                                blend_mode: str = "average",
                                seed: int = 42) -> dict:
        """Generate hybrid creature from two images"""

        try:
            # Auto-caption if descriptions not provided
            if not desc1:
                desc1 = self.auto_caption_image(image1)
            if not desc2:
                desc2 = self.auto_caption_image(image2)

            # Preprocess images
            image1 = image1.convert('RGB').resize((512, 512), Image.LANCZOS)
            image2 = image2.convert('RGB').resize((512, 512), Image.LANCZOS)

            # Blend edge maps
            control_image = self.hybrid_generator.blend_edge_maps(image1, image2, blend_mode)

            # Generate hybrid
            style = self.style_library.styles.get(style_key, self.style_library.styles["moebius"])
            hybrid_prompt = f"{prompt} combining {desc1} and {desc2} {style['prompt_suffix']}"

            generator = torch.Generator(device="cuda").manual_seed(seed)

            result = self.pipe(
                prompt=hybrid_prompt,
                image=control_image,
                negative_prompt=style["negative_prompt"],
                num_inference_steps=25,
                guidance_scale=7.5,
                width=512,
                height=512,
                controlnet_conditioning_scale=style["conditioning_scale"],
                generator=generator
            )

            hybrid_image = result.images[0]

            # Create labeled version
            labeled_image = self.hybrid_generator.create_component_labels(desc1, desc2, hybrid_image)

            # Generate backstory
            creature_name = f"{desc1.split()[0]}-{desc2.split()[0]}".title()
            backstory = self.style_library.get_backstory(
                creature_name, style_key, [desc1, desc2]
            )

            return {
                "hybrid_image": hybrid_image,
                "labeled_image": labeled_image,
                "control_image": control_image,
                "backstory": backstory,
                "components": [desc1, desc2],
                "blend_mode": blend_mode
            }

        except Exception as e:
            print(f"❌ Hybrid generation error: {e}")
            return self._create_error_response(str(e))

    def _create_error_response(self, error_msg: str) -> dict:
        """Create error response with visual feedback"""
        error_img = Image.new('RGB', (512, 512), color=(40, 20, 20))
        draw = ImageDraw.Draw(error_img)
        draw.text((50, 200), "AI Circus Error", fill=(255, 100, 100))
        draw.text((50, 230), error_msg[:50], fill=(255, 150, 150))

        return {
            "creature_image": error_img,
            "backstory": f"Error: {error_msg[:100]}",
            "error": True
        }

# ====================
# 5. GRADIO INTERFACE
# ====================

def create_enhanced_interface():
    """Create the enhanced Gradio interface"""

    circus = EnhancedAICircus()

    with gr.Blocks(theme=gr.themes.Soft(), title="🎪 AI Circus") as demo:

        gr.Markdown("""
        # 🎪 AI Circus of Strange Forms
        ## *Interactive GIFs × Artistic Styles × Hybrid Creatures*

        **Transform images into animated creatures with artistic styles and narratives**
        """)

        with gr.Tabs() as tabs:

            # TAB 1: SINGLE IMAGE TO ANIMATED CREATURE
            with gr.TabItem("🎭 Animated Creature Generator"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### 📸 Upload & Configure")

                        single_image = gr.Image(
                            label="Upload Base Image",
                            type="pil",
                            height=256,
                            sources=["upload", "webcam"]
                        )

                        single_prompt = gr.Textbox(
                            label="Creature Description",
                            value="a surreal biomechanical creature",
                            lines=2
                        )

                        style_dropdown = gr.Dropdown(
                            choices=list(circus.style_library.styles.keys()),
                            value="moebius",
                            label="🎨 Artistic Style",
                            info="Select an artist's style"
                        )

                        with gr.Row():
                            bend_slider = gr.Slider(
                                0.0, 1.0, 0.3,
                                label="🌀 Network Bending",
                                info="How much to distort the AI"
                            )
                            seed_input = gr.Slider(
                                0, 100000, 4242,
                                label="🎲 Random Seed"
                            )

                        generate_gif = gr.Checkbox(
                            value=True,
                            label="🎬 Generate 5-second GIF",
                            info="Create animated version"
                        )

                        generate_btn = gr.Button(
                            "🎪 Create Animated Creature!",
                            variant="primary",
                            size="lg"
                        )

                    with gr.Column(scale=2):
                        output_gallery = gr.Gallery(
                            label="Generated Outputs",
                            height=500,
                            columns=2
                        )

                        backstory_output = gr.Textbox(
                            label="📖 Creature Backstory",
                            lines=3,
                            interactive=False
                        )

                        style_info = gr.Textbox(
                            label="🎨 Style Info",
                            value="",
                            interactive=False
                        )

                        gif_output = gr.File(
                            label="🎬 Animated GIF",
                            visible=False
                        )

                # Update style info when dropdown changes
                def update_style_info(style_key):
                    style = circus.style_library.styles.get(style_key, {})
                    return f"Selected: {style.get('name', 'Unknown')}\nColors: {style.get('color_palette', 'Various')}"

                style_dropdown.change(
                    fn=update_style_info,
                    inputs=style_dropdown,
                    outputs=style_info
                )

                # Generate function for single image
                def generate_single_creature(image, prompt, style_key, bend, seed, make_gif):
                    if image is None:
                        return [], "Please upload an image first", "", None

                    result = circus.generate_creature(
                        image, prompt, style_key, bend, seed, make_gif
                    )

                    outputs = [result["creature_image"], result["control_image"]]

                    if make_gif and result["gif_path"]:
                        gif_file = result["gif_path"]
                    else:
                        gif_file = None

                    return outputs, result["backstory"], update_style_info(style_key), gif_file

                generate_btn.click(
                    fn=generate_single_creature,
                    inputs=[single_image, single_prompt, style_dropdown, bend_slider, seed_input, generate_gif],
                    outputs=[output_gallery, backstory_output, style_info, gif_output]
                ).then(
                    fn=lambda: gr.update(visible=True),
                    outputs=[gif_output]
                )

            # TAB 2: HYBRID CREATURE CREATOR
            with gr.TabItem("🧬 Hybrid Creature Fusion"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### 🧬 Upload Two Images")

                        with gr.Row():
                            hybrid_image1 = gr.Image(
                                label="First Image",
                                type="pil",
                                height=200
                            )
                            hybrid_image2 = gr.Image(
                                label="Second Image",
                                type="pil",
                                height=200
                            )

                        with gr.Row():
                            desc1 = gr.Textbox(
                                label="Description of First",
                                value="cat",
                                placeholder="e.g., cat, robot, tree"
                            )
                            desc2 = gr.Textbox(
                                label="Description of Second",
                                value="robot",
                                placeholder="e.g., dog, angel, mushroom"
                            )

                        hybrid_prompt = gr.Textbox(
                            label="Hybrid Description",
                            value="a hybrid creature",
                            lines=2
                        )

                        hybrid_style = gr.Dropdown(
                            choices=list(circus.style_library.styles.keys()),
                            value="h_r_giger",
                            label="🎨 Fusion Style"
                        )

                        blend_mode = gr.Radio(
                            choices=["average", "maximum", "interlace"],
                            value="average",
                            label="🧬 Blend Mode",
                            info="How to combine the images"
                        )

                        hybrid_seed = gr.Slider(
                            0, 100000, 7777,
                            label="🎲 Fusion Seed"
                        )

                        create_hybrid_btn = gr.Button(
                            "🧬 Create Hybrid Creature!",
                            variant="primary",
                            size="lg"
                        )

                    with gr.Column(scale=2):
                        hybrid_gallery = gr.Gallery(
                            label="Hybrid Creation",
                            height=500,
                            columns=2
                        )

                        hybrid_backstory = gr.Textbox(
                            label="📖 Hybrid Backstory",
                            lines=4,
                            interactive=False
                        )

                        component_info = gr.Textbox(
                            label="🧬 Component Origins",
                            interactive=False
                        )

                # Generate hybrid function
                def create_hybrid(img1, img2, d1, d2, prompt, style, blend, seed):
                    if img1 is None or img2 is None:
                        return [], "Upload both images", ""

                    result = circus.generate_hybrid_creature(
                        img1, img2, d1, d2, prompt, style, blend, seed
                    )

                    outputs = [result["hybrid_image"], result["labeled_image"], result["control_image"]]
                    components = f"Combining: {d1} + {d2} (Mode: {blend})"

                    return outputs, result["backstory"], components

                create_hybrid_btn.click(
                    fn=create_hybrid,
                    inputs=[hybrid_image1, hybrid_image2, desc1, desc2, hybrid_prompt, hybrid_style, blend_mode, hybrid_seed],
                    outputs=[hybrid_gallery, hybrid_backstory, component_info]
                )

            # TAB 3: ARTISTIC STYLE GALLERY
            with gr.TabItem("🎨 Style Gallery & Examples"):
                gr.Markdown("### 🎨 Artistic Style Reference")

                style_examples = []
                for style_key, style_info in circus.style_library.styles.items():
                    style_examples.append(
                        f"### {style_info['name']}\n"
                        f"**Key words:** {style_info['prompt_suffix'][:100]}...\n"
                        f"**Colors:** {style_info['color_palette']}\n"
                        f"**Best for:** {style_info.get('negative_prompt', 'Various').replace('realistic, ', '')}\n"
                        "---"
                    )

                style_guide = gr.Markdown("\n\n".join(style_examples))

                gr.Markdown("""
                ### 💡 Prompt Tips:
                - **Dalí:** Use words like "melting", "dream", "surreal", "distorted"
                - **Moebius:** Use "geometric", "clean lines", "vibrant", "sci-fi"
                - **Miyazaki:** Use "whimsical", "magical", "soft", "friendly"
                - **Giger:** Use "biomechanical", "alien", "organic machinery", "dark"
                - **Van Gogh:** Use "brushstrokes", "swirling", "expressive", "textured"
                - **Bosch:** Use "grotesque", "intricate", "medieval", "nightmarish"
                """)

        # Footer
        gr.Markdown("""
        ---
        **🎯 Try These Examples:**
        - a biomechanical creature with crystal growth
        - a furry monster with multiple eyes
        - a robotic animal with glowing patterns
        - a surreal hybrid with plant-like features

        ---
        ### 🎪 AI Circus Features:
        - **Interactive GIFs:** 5-second animated creatures
        - **Artistic Styles:** 6 classic artist styles with auto-backstories
        - **Hybrid Creation:** Fuse two images into one creature
        - **Network Bending:** Controlled distortion of AI generations

        *Built with Stable Diffusion + ControlNet + Network Bending*
        """)

        return demo

# ====================
# 6. TEST & LAUNCH
# ====================

def run_tests():
    """Test the system"""
    print("🧪 Testing Enhanced AI Circus...")

    circus = EnhancedAICircus()

    # Test styles
    print("🎨 Available styles:")
    for key, style in circus.style_library.styles.items():
        print(f"  • {style['name']}")

    # Test backstory generation
    test_backstory = circus.style_library.get_backstory("Crystal Deer", "miyazaki")
    print(f"📖 Sample backstory: {test_backstory}")

    print("✅ Tests completed!")
    return circus

# Launch the interface
if __name__ == "__main__":
    print("🚀 Launching Enhanced AI Circus...")

    # Run quick tests
    run_tests()

    # Create and launch interface
    interface = create_enhanced_interface()

    # Launch with better settings for Colab
    interface.launch(
        share=True,
        debug=False,
        server_name="0.0.0.0",
        server_port=7860,
        quiet=True,
        show_error=True
    )